In [1]:
from pymongo import MongoClient
client = MongoClient()
db = client.mydisease.ctdbase
db.drop()

In [2]:
import gzip
import json
import os
from collections import defaultdict
from tqdm import tqdm
import pandas as pd

from mydisease import DATA_DIR
from mydisease.dataload.ctdbase.download_raw_data import relationships
pd.set_option('display.width', 1000)

In [3]:
def parse_diseaseid(did: str):
    """ 
    The 'DiseaseID' column sometimes starts with the identifier prefix, and sometime doesnt
    prefixes are {'MESH:','OMIM:'}
    if an ID starts with 'C' or 'D', its MESH, if its an integer: 'OMIM'
    """
    if did.startswith("OMIM:") or did.startswith("MESH:"):
        return did.split(":",1)[0].lower() + ":" + did.split(":",1)[1]
    if did.startswith('C') or did.startswith('D'):
        return 'mesh:' + did
    if did.isdigit():
        return "omim:" + did
    raise ValueError(did)
    return did

In [4]:
def parse_csv_to_df(f):
    line = next(f)
    while not line.startswith("# Fields:"):
        line = next(f)
    # parse the column headers from the comments
    fields = next(f)[1:].strip().split(",")
    df = pd.read_csv(f, delimiter=",", comment="#")
    df.columns = fields

    # split pipe-delimited fields
    fields_split = set(['DirectEvidence', 'OmimIDs', 'PubMedIDs', 'InferenceGeneSymbols']) & set(fields)
    for field in fields_split:
        # don't split NaN
        field_split = df[field].dropna().astype(str).str.split("|")
        df[field][field_split.index] = field_split

    df['DiseaseID'] = df['DiseaseID'].map(parse_diseaseid)
    return df

In [5]:
def get_columns_to_keep(relationship):
    if relationship in {'GO_BP', 'GO_CC', 'GO_MF'}:
        columns_keep = ['GOID','InferenceGeneSymbols']
    elif relationship == "pathways":
        columns_keep = ['PathwayID','InferenceGeneSymbol']
    elif relationship == "chemicals":
        columns_keep = ['CasRN','ChemicalID','DirectEvidence','InferenceGeneSymbol','InferenceScore','OmimIDs','PubMedIDs']
    elif relationship == "genes":
        columns_keep = ['GeneID','DirectEvidence','InferenceScore','InferenceChemicalName','OmimIDs','PubMedIDs']
    return columns_keep

In [6]:
def parse_df(db, df, relationship):
    """
    df is parsed and added to mongodb (db)
    """
    columns_keep = get_columns_to_keep(relationship)
    total = len(set(df.DiseaseID))
    for diseaseID, subdf in tqdm(df.groupby("DiseaseID"), total=total):
        sub = subdf[columns_keep].to_dict(orient="records")
        sub = [{k:v for k,v in s.items() if v==v} for s in sub] # get rid of nulls
        db.update_one({'_id':diseaseID}, {'$set':{relationship: sub}}, upsert=True)

In [7]:
def process_genes(db, f):
    """
    # for the genes file, which is enormous, we need to do something different
    # basically same as others, but in chunks
    d is modified in place!!
    
    note: this will fail
    WriteError: Resulting document after update is larger than 16777216

    """
    chunksize = 100000
    names = ['GeneSymbol', 'GeneID', 'DiseaseName', 'DiseaseID', 'DirectEvidence',
             'InferenceChemicalName', 'InferenceScore', 'OmimIDs', 'PubMedIDs']
    for df in tqdm(pd.read_csv(f, delimiter=",", comment="#", header=None, chunksize=chunksize, 
                          low_memory=False, names=names), total = 49867785/chunksize):
        fields_split = ['DirectEvidence', 'OmimIDs', 'PubMedIDs']
        for field in fields_split:
            field_split = df[field].dropna().astype(str).str.split("|")
            df[field][field_split.index] = field_split
        columns_keep = get_columns_to_keep('genes')
        df['DiseaseID'] = df['DiseaseID'].map(parse_diseaseid)
        for diseaseID, subdf in df.groupby("DiseaseID"):
            sub = subdf[columns_keep].to_dict(orient="records")
            # get rid of nulls
            sub = [{k:v for k,v in s.items() if v==v} for s in sub]
            db.update_one({'_id':diseaseID}, {'$push':{relationship: {'$each': sub}}}, upsert=True)

In [8]:
for relationship, file_path in relationships.items():
    print(relationship)
    with gzip.open(os.path.join(DATA_DIR, file_path), 'rt', encoding='utf-8') as f:
        if relationship == "genes":
            #process_genes(db, f)
            print("skipping genes")
            pass
        else:
            df = parse_csv_to_df(f)
            parse_df(db, df, relationship)

pathways


100%|██████████| 3556/3556 [00:05<00:00, 685.10it/s]


GO_BP


/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
100%|██████████| 4298/4298 [00:15<00:00, 272.25it/s]


GO_MF


100%|██████████| 4281/4281 [00:09<00:00, 439.85it/s]


GO_CC


100%|██████████| 4297/4297 [00:07<00:00, 596.64it/s]


genes
skipping genes
chemicals


100%|██████████| 5735/5735 [02:12<00:00, 43.23it/s] 


In [9]:
db.count()

5749

In [13]:
doc = db.find_one({'genes':{'$exists':True}})
doc

In [14]:
from collections import Counter
Counter([x['_id'][:6] for x in db.find({},{'_id':1})])

Counter({'mesh:C': 2040,
         'mesh:D': 3327,
         'omim:1': 70,
         'omim:2': 62,
         'omim:3': 27,
         'omim:4': 2,
         'omim:6': 221})

In [16]:
db.find_one('mesh:D006954')

{'GO_BP': [{'GOID': 'GO:0055090', 'InferenceGeneSymbols': ['APOA5']},
  {'GOID': 'GO:0042632', 'InferenceGeneSymbols': ['APOA5']},
  {'GOID': 'GO:0006869', 'InferenceGeneSymbols': ['APOA5']},
  {'GOID': 'GO:0042157', 'InferenceGeneSymbols': ['APOA5']},
  {'GOID': 'GO:0031100', 'InferenceGeneSymbols': ['APOA5']},
  {'GOID': 'GO:0045723', 'InferenceGeneSymbols': ['APOA5']},
  {'GOID': 'GO:0050996', 'InferenceGeneSymbols': ['APOA5']},
  {'GOID': 'GO:0051006', 'InferenceGeneSymbols': ['APOA5']},
  {'GOID': 'GO:0048260', 'InferenceGeneSymbols': ['APOA5']},
  {'GOID': 'GO:0010898', 'InferenceGeneSymbols': ['APOA5']},
  {'GOID': 'GO:0010902', 'InferenceGeneSymbols': ['APOA5']},
  {'GOID': 'GO:0009725', 'InferenceGeneSymbols': ['APOA5']},
  {'GOID': 'GO:0042246', 'InferenceGeneSymbols': ['APOA5']},
  {'GOID': 'GO:0006810', 'InferenceGeneSymbols': ['APOA5']},
  {'GOID': 'GO:0019433', 'InferenceGeneSymbols': ['APOA5']},
  {'GOID': 'GO:0070328', 'InferenceGeneSymbols': ['APOA5']},
  {'GOID': 'GO: